In [1]:
from mistralai import Mistral
import pandas as pd
import minsearch
import os
from groq import Groq
from dotenv import load_dotenv
from tqdm.auto import tqdm
import pandas as pd
import minsearch
import os
from groq import Groq
from dotenv import load_dotenv
from tqdm.auto import tqdm


## Environment variables Loading

In [2]:
load_dotenv()


True

In [3]:
# Access the API keys
mistral_api_key = os.getenv("MISTRAL_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

In [4]:
%whos

Variable          Type        Data/Info
---------------------------------------
Groq              type        <class 'groq.Groq'>
Mistral           type        <class 'mistralai.sdk.Mistral'>
groq_api_key      str         gsk_32VwUDXWlbcss91AZEhIW<...>Yv5D56tMak8RwoyfSBXePPdht
load_dotenv       function    <function load_dotenv at 0x00000145257C5310>
minsearch         module      <module 'minsearch' from <...>notebooks\\minsearch.py'>
mistral_api_key   str         8liS0F7zb7zNgujwgo3dgTbr6HbfvmeI
os                module      <module 'os' from 'C:\\Us<...>n\\Python39\\lib\\os.py'>
pd                module      <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
tqdm              type        <class 'tqdm.auto.tqdm'>


In [5]:
df=pd.read_csv("../dataset/Mental_Health_FAQ.csv")

In [6]:
df.head()

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [7]:
documents=df.to_dict(orient='records')

In [107]:
documents

[{'Question_ID': 1590140,
  'Questions': 'What does it mean to have a mental illness?',
  'Answers': 'Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.\nIt is important to know that mental illnesses are medical conditions that have nothing to do with a personâ€™s character, intelligence, or willpower. Just as diabetes is a disorder of the pancreas, mental illness is a medical con

In [ ]:
prompt_template="""You are emulating a patient who is concerned about mental health.
Based on the answer provided, formulate 3 questions that this patient might ask. 
The questions should be complete, not too short, and use as few words as possible from the original answer.

The record:
Question_ID:{Question_ID}
Questions: {Questions}
Answer: {Answers}

Provide the output in parsable JSON format without using code blocks:

{{["question1", "question2",..., "question5"]}}

""".strip()

In [80]:
prompt=prompt_template.format(**documents[0])

In [120]:
client = Groq(api_key=groq_api_key)

In [81]:
def llm(prompt,model="mixtral-8x7b-32768"):
        response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}])
        return response.choices[0].message.content
    

In [61]:
quiz=llm(prompt)

In [62]:
print(quiz)

[
"How do mental illnesses affect a person's daily life?",
"What are some examples of serious mental illnesses?",
"What should I understand about the cause of mental illness?",
"What are the common treatments for mental illness?",
"How effective are the treatments for mental illness and what is the quality of life people can expect after receiving treatment?"
]


In [16]:
len(quiz)

666

In [121]:
import json

prompt_template="""You are emulating a patient who is concerned about mental health.
Based on the answer provided, formulate 3 questions that this patient might ask. 
The questions should be complete, not too short, and use as few words as possible from the original answer.

The record:
Question_ID:{Question_ID}
Questions: {Questions}
Answer: {Answers}

Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}

""".strip()
def generate_data(doc):
    prompt=prompt_template.format(**doc)
    response = client.chat.completions.create(
        model='mixtral-8x7b-32768',
        messages=[{"role": "user", "content": prompt}])
    return response.choices[0].message.content
    

In [122]:
results={}
for doc in tqdm(documents):
    doc_id=doc['Question_ID']
    if doc_id in results:
        continue
    questions_raw = generate_data(doc)
    print(f"Raw JSON data for doc_id {doc_id}: {questions_raw}")
    
    # Check if it's empty
    if not questions_raw:
        print(f"Error: Empty response for doc_id {doc_id}")
    else:
        # Try parsing it
        try:
            questions = json.loads(questions_raw)
            results[doc_id] = questions['questions']
        except json.JSONDecodeError as e:
            print(f"JSON decoding failed for doc_id {doc_id}: {e}")


  0%|          | 0/98 [00:00<?, ?it/s]

Raw JSON data for doc_id 1590140: {
"questions": [
"How do mental illnesses affect a person's daily functioning and relationships?",
"What are some examples of serious mental illnesses that can result in major functional impairment?",
"Why is it a misconception to associate mental illness with a person's character, intelligence, or willpower?",
"How are mental illnesses treated, and what is the success rate of these treatments?",
"Can a person with mental illness become independent and successful with proper treatment?"
]
}
Raw JSON data for doc_id 2110618: {
"questions": [
"Who are the groups most commonly affected by different mental illnesses?",
"How do mental illnesses affect children and the elderly?",
"What behaviors should parents and caretakers watch out for in children that may indicate a serious emotional disturbance?"
]
}
Raw JSON data for doc_id 6361820: {
"questions": [
"How does mental illness affect different populations?",
"At what age are individuals most susceptible t

In [123]:
final_results = []

for doc_id, questions in results.items():
    for q in questions:
        final_results.append((doc_id, q))


In [128]:
final_results[10]


(6361820,
 'What signs should parents and caretakers look out for in children under 18 that may indicate a mental health condition?')

In [129]:
df_results = pd.DataFrame(final_results, columns=['id', 'question'])


In [131]:
df_results.head(10)

,id,question
0,1590140,How do mental illnesses affect a person's dail...
1,1590140,What are some examples of serious mental illne...
2,1590140,Why is it a misconception to associate mental ...
3,1590140,"How are mental illnesses treated, and what is ..."
4,1590140,Can a person with mental illness become indepe...
5,2110618,Who are the groups most commonly affected by d...
6,2110618,How do mental illnesses affect children and th...
7,2110618,What behaviors should parents and caretakers w...
8,6361820,How does mental illness affect different popul...
9,6361820,At what age are individuals most susceptible t...


In [133]:
df_results.to_csv('../dataset/ground_truth_data.csv', index=False)
